In [61]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf 

#### Задание 1
Загрузите данные, проверьте типы, пропущенные значения. 

Сколько пропущенных значений встретилось в датасете?

In [2]:
df = pd.read_csv('cars.csv')

In [20]:
df.isna().sum().sum()

0

#### Задание 2
Сгенерируем первый признак.

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент. Например:

'audi 100 ls' → 'audi'
Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.


In [16]:
df['company'] = df.CarName.str.split(' ').str[0]

In [22]:
df = df.drop(columns=['car_ID','CarName'])

In [23]:
df.company.nunique()

28

#### Задание 3
Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!

* 'maxda' → 'mazda'
* 'Nissan' → 'nissan'
* 'porcshce' → 'porsche'
* 'toyouta' → 'toyota'
* 'vokswagen' & 'vw' → 'volkswagen'

Сколько уникальных производителей осталось в итоге?

In [28]:
df.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'volkswagen',
       'volvo'], dtype=object)

In [30]:
df['company'] = df.company.str.lower()

to_replace = {
    'toyouta':'toyota',
    'vokswagen' : 'volkswagen',
    'vw' :'volkswagen',
    'maxda' : 'mazda',
    'porcshce' : 'porsche'
}

for typo in to_replace:
    df.loc[df.company == typo, 'company'] = to_replace[typo]

In [31]:
df.company.nunique()

22

#### Задание 4
Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

<pre>
'company', 'fueltype', 'aspiration','carbody',
'drivewheel', 'wheelbase', 'carlength','carwidth',
'curbweight', 'enginetype', 'cylindernumber', 'enginesize',
'boreratio','horsepower'
</pre>
также не забыв про то, что мы предсказываем – 'price'. 

После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? Ответ округлите до 2 знаков после точки.

Hint: можно использовать метод для датафрейма – .corr()

In [34]:
interested = ['price','company', 'fueltype', 'aspiration','carbody',
'drivewheel', 'wheelbase', 'carlength','carwidth',
'curbweight', 'enginetype', 'cylindernumber', 'enginesize',
'boreratio','horsepower']

In [108]:
df_ = df[interested]

In [109]:
df_.corr().round(2)

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower
price,1.00,0.58,0.68,0.76,0.84,0.87,0.55,0.81
wheelbase,0.58,1.00,0.87,0.80,0.78,0.57,0.49,0.35
carlength,0.68,0.87,1.00,0.84,0.88,0.68,0.61,0.55
carwidth,0.76,0.80,0.84,1.00,0.87,0.74,0.56,0.64
curbweight,0.84,0.78,0.88,0.87,1.00,0.85,0.65,0.75
enginesize,0.87,0.57,0.68,0.74,0.85,1.00,0.58,0.81
boreratio,0.55,0.49,0.61,0.56,0.65,0.58,1.00,0.57
horsepower,0.81,0.35,0.55,0.64,0.75,0.81,0.57,1.00


#### Задание 5
Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies(). Пример использования:

<code>
df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)
</code>

Не забудьте присоединить к полученному датасету столбцы с переменными других типов :) Также обратите внимание: мы должны использовать тот же набор колонок, который был на предыдущем шаге!

Сколько колонок теперь имеется в датасете?

Hint: pd.concat([df_1, df_2], axis=1)

In [99]:
df_.head()

,price,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower
0,13495.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
1,16500.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
2,16500.0,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154
3,13950.0,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102
4,17450.0,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115


In [110]:
df_.shape

(205, 15)

In [100]:
to_dummies = df_.dtypes[df_.dtypes == 'object'].index.values

In [111]:
df_dummy = pd.concat([df_.drop(columns=to_dummies), pd.get_dummies(data=df_[to_dummies], drop_first=True)], axis=1)

In [113]:
df_dummy.shape

(205, 49)

In [112]:
len(df_dummy.columns)

49

#### Задание 6
Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

Какой процент изменчивости объясняет полученная модель? (округлите до целого)

In [114]:
results = smf.ols('price ~ horsepower', data=df_dummy).fit()
summ = results.summary()
print(summ)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Sat, 25 Jun 2022   Prob (F-statistic):           1.48e-48
Time:                        02:00:55   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

In [104]:
round(0.651 * 100, 0)

65.0

#### Задание 7
Теперь – две модели:

* модель со всеми предикторами
* модель со всеми предикторами, кроме марок машин

Обратите внимание на изменения в R^2, коэффициентах и их значимости. Какую модель лучше оставить?
* Где больше предикторов, там R2 самый большой
* Вообще с одним предиктором – horsepower!
* **Где меньше предикторов, ведь R2 изменился не очень сильно, а часть марок вообще не значима**

In [115]:
all_predictors = df_dummy.columns[df_dummy.columns != 'price'].values

In [123]:
X = sm.add_constant(df_dummy[all_predictors])  # добавить константу, чтобы был свободный член
Y = df.price
model = sm.OLS(Y, X)  # говорим модели, что у нас ЗП, а что НП
results = model.fit()  # строим регрессионную прямую
all_predictors_summ = results.summary()
print(all_predictors_summ)  # смотрим результат

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Sat, 25 Jun 2022   Prob (F-statistic):           4.86e-89
Time:                        02:11:21   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

In [173]:
# Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
results_as_html = all_predictors_summ.tables[1].as_html()
df_reg = pd.read_html(results_as_html, header=0, index_col=0)[0]
df_reg.head()

,coef,std err,t,P>|t|,[0.025,0.975]
const,-34720.0000,11700.000,-2.959,0.004,-57900.000,-11500.000
wheelbase,147.1396,76.234,1.930,0.055,-3.429,297.708
carlength,-113.2012,48.949,-2.313,0.022,-209.881,-16.522
carwidth,809.1860,214.508,3.772,0.000,385.514,1232.858
curbweight,4.1822,1.582,2.643,0.009,1.057,7.307


In [175]:
(df_reg['P>|t|'] > 0.05).sum()

33

In [119]:
predictors = df_dummy.columns[df_dummy.columns != 'price']
predictors = predictors[~predictors.str.contains('company')].values

In [176]:
X = sm.add_constant(df_dummy[predictors])  # добавить константу, чтобы был свободный член
Y = df.price
model = sm.OLS(Y, X)  # говорим модели, что у нас ЗП, а что НП
results = model.fit()  # строим регрессионную прямую
summ = results.summary() 
print(summ)  # смотрим результат

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Sat, 25 Jun 2022   Prob (F-statistic):           9.86e-81
Time:                        02:30:07   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+

#### Задание 8
Заполните пропуски:



In [180]:
len(predictors)

27

In [184]:
results_as_html = summ.tables[1].as_html()
df_reg = pd.read_html(results_as_html, header=0, index_col=0)[0]
df_reg

,coef,std err,t,P>|t|,[0.025,0.975]
const,-17000.0000,13000.000,-1.309,0.192,-42600.000,8625.219
wheelbase,71.1868,87.028,0.818,0.414,-100.552,242.925
carlength,-51.3497,50.341,-1.020,0.309,-150.692,47.993
carwidth,541.8700,253.327,2.139,0.034,41.958,1041.782
curbweight,2.9577,1.796,1.647,0.101,-0.585,6.501
enginesize,36.0515,22.376,1.611,0.109,-8.105,80.208
boreratio,-2230.4519,1731.681,-1.288,0.199,-5647.719,1186.815
horsepower,86.8164,16.717,5.193,0.000,53.827,119.806
fueltype_gas,-2423.0935,975.579,-2.484,0.014,-4348.283,-497.904
aspiration_turbo,-1139.2334,787.230,-1.447,0.150,-2692.738,414.271


In [191]:
# исключаем const
(df_reg['P>|t|'][1:] > 0.05).sum()

10